# Notebook Overview

1. **Setup**:
    - Import libraries and configure Polars settings.

2. **Data Loading**:
    - Load predictions, entries, stops, and patterns data from Parquet and JSON files.

3. **Data Mapping and Formatting**:
    - Define mappings for route IDs and status colors.
    - Format and correct time columns in predictions data.

4. **Data Processing**:
    - Join stops data to entries data to get stop names.
    - Filter and transform entries data to include only in-service records and calculate time differences.

5. **Subset and Join Operations**:
    - Filter data for specific route IDs and stops.
    - Perform join operations to calculate time differences for stops ahead.

6. **Descriptive Statistics**:
    - Generate and display descriptive statistics for time differences.


In [ ]:
import polars as pl
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import json
import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pl.enable_string_cache()
pl.Config().set_tbl_cols(100)
pl.Config().set_tbl_rows(25)

In [ ]:
%config InteractiveShell.ast_node_interactivity = 'last_expr_or_assign'

In [ ]:
#Map route id to correct route name
route_mapping = {
    3: "2L",
    4: "2R",
    33: "3",
    17: "10",
    18: "11",
    23: "12",
    12: "16",
    13: "17",
    14: "18",
    30: "19",
    29: "21",
    38: "21 Tripper",
    777: "777"
}

#Map hex color to prediction status
color_mapping = {
    "#cf1625": "Early",
    "#39B139": "On Time",
    "#D58803": "Little Late",
    "#f3e413": "Very Late"
}

In [ ]:
preds = pl.read_parquet("./data/2024-09-preds.parquet")

preds = preds.with_columns(pl.col("statuscolor").alias("statusName"))
preds = preds.with_columns(pl.col("statusName").replace_strict(color_mapping))

#Format the schedule time to be a datetime object
preds = preds.with_columns(
    pl.col('receiveTime').dt.convert_time_zone('America/Chicago'),
    pl.col('schedule').alias('scheduleStr'),
    pl.col('time').alias('timeStr'),
    pl.col('schedule').str.to_time(format='%I:%M%p', strict=False),
    pl.col('time').str.to_time(format='%I:%M%p', strict=False)
)

preds = preds.with_columns(pl.col('receiveTime').dt.combine(pl.col('time')).alias('time'),
                    pl.col('receiveTime').dt.combine(pl.col('schedule')).alias('schedule'))

#Correct for time columns hat are off by 24 hours
preds = preds.with_columns(
    pl.when(pl.col('time') - pl.col('receiveTime') > pl.duration(hours=12))
    .then(pl.col('time') - pl.duration(hours=24))
    .when(pl.col('time') - pl.col('receiveTime') < pl.duration(hours=-12))
    .then(pl.col('time') + pl.duration(hours=24))
    .otherwise(pl.col('time'))
    .alias('time')
)

#Correct for schedule times columns that are off by 24 hours
preds = preds.with_columns(
    pl.when(pl.col('schedule') - pl.col('receiveTime') > pl.duration(hours=12))
    .then(pl.col('schedule') - pl.duration(hours=24))
    .when(pl.col('schedule') - pl.col('receiveTime') < pl.duration(hours=-12))
    .then(pl.col('schedule') + pl.duration(hours=24))
    .otherwise(pl.col('schedule'))
    .alias('schedule')
)

# preds = preds.with_columns(
#     (pl.col("time") - pl.col("receiveTime")).alias("predictedDiff")
# )

# preds = preds.with_columns(
#     (pl.col("schedule") - pl.col("receiveTime")).alias("scheduleDiff")
# )


In [ ]:
#preds['scheduleDiff'].describe()

In [ ]:
#preds['predictedDiff'].describe()

In [ ]:
df = pl.read_parquet("./data/2024-09-entries.parquet")
df = df.with_columns(pl.col("routeID").replace_strict(route_mapping))


In [ ]:
#Load stops data
file = open("./data/stops.json", "r")
stopsData = json.load(file)

stops = pl.DataFrame(stopsData['get_stops'])

In [ ]:
#Load patterns data
pattern_mapping = {
    3: "2L",
    4: "2R",
    37: "3",
    17: "10",
    18: "11",
    23: "12",
    12: "16",
    13: "17",
    14: "18",
    33: "19",
    46: "21",
    45: "21 Tripper",
}

#Load patterns json
file = open("./data/patterns.json", "r")
patternsData = json.load(file)

patterns = pl.DataFrame(patternsData['get_patterns'])
patterns = patterns.with_columns(pl.col("id").replace_strict(pattern_mapping, default="None"))

In [ ]:
stops.rename({"id": "nextStopID"}).select(["nextStopID", "name"]).unique().sort("nextStopID")

In [ ]:
#Left join stops to get stop names for nextStopID and lastStopID
df = df.join(stops.rename({"id": "nextStopID"}).select(["nextStopID", "name"]).unique(), on="nextStopID", how="left").rename({"name": "nextStopName"})
df = df.join(stops.rename({"id": "lastStopID"}).select(["lastStopID", "name"]).unique(), on="lastStopID", how="left").rename({"name": "lastStopName"})


In [ ]:
#Filter out the subset of data we want to work with
df = df.filter(
    (pl.col("inService"))
)

#Add stopChanged column
df = df.with_columns(
    (
        (pl.col("lastStopID") == pl.col("nextStopID").shift(1)).over(
            "equipmentID", order_by="receiveTime"
        )
    ).alias("stopChanged")
)

df = df.filter(pl.col("stopChanged")).with_columns(
    (pl.col("nextStopID").shift(1) == pl.col("lastStopID"))
    .over("equipmentID", order_by="receiveTime")
    .alias("nextToLast")
)

#Add timeDiff column
df = df.with_columns(
    (-pl.col("receiveTime").diff(-1).over("equipmentID", order_by="receiveTime")).alias(
        "timeDiff"
    )
).filter(pl.col("nextToLast"))


In [ ]:
subset = df.filter(
    pl.col("routeID") == "2L"
)

In [ ]:
df1 = subset.filter(pl.col('nextStopID') == 465)
df2 = subset.filter(pl.col('nextStopID') == 468)

df2 = df2.with_columns(pl.col("receiveTime").alias("receiveTime_right"))
joined_df = df1.join_asof(df2, on="receiveTime", by='equipmentID', strategy='forward')

joined_df = joined_df.with_columns(
    (pl.col("receiveTime_right") - pl.col("receiveTime")).alias("timeDiff_3_stops_ahead")
)

joined_df['routeID', 'equipmentID', 'lastStopName', 'nextStopName_right', 'receiveTime', 'receiveTime_right', 'timeDiff_3_stops_ahead']

In [ ]:
mega_df = pl.read_parquet('./data/mega_df.parquet')

In [ ]:
mega_df['routeID'].unique()

In [ ]:
mega_df['routeID', 'equipmentID', 'lastStopID', 'lastStopID_right', 'lastStopName', 'lastStopName_right',  'receiveTime', 'receiveTime_right', 'eta']

In [ ]:
m_pred_stopIds = mega_df['lastStopID', 'lastStopID_right'].unique().filter(pl.col('lastStopID_right').is_not_null())

In [ ]:
#Join the subset on the triple key to get the correct prediction
# m_predset = subset.join(
#     preds,
#     on=["equipmentID", "captureTime"],
#     how="inner"
# )

# m_predset = m_predset['lastStopID', 'stopID'].unique()

# m_pred_stopIds = m_pred_stopIds.rename({"lastStopID_right": "stopID"})

# anti_join_set = m_pred_stopIds.join(
#     m_predset,
#     on=["lastStopID", "stopID"],
#     how="anti"
# )

# anti_join_set = anti_join_set.join(stops.rename({"id": "stopID"}).select(["stopID", "name"]).unique(), on="stopID", how="left").rename({"name": "nextStopName"})
# anti_join_set = anti_join_set.join(stops.rename({"id": "lastStopID"}).select(["lastStopID", "name"]).unique(), on="lastStopID", how="left").rename({"name": "lastStopName"})

# anti_join_set = anti_join_set.sort(by='lastStopID')

In [ ]:
subset['timeDiff'].describe()

In [ ]:
mega_predset = mega_df.join(
    preds,
    left_on=["equipmentID", "captureTime", 'nextStopID_actual'],
    right_on=["equipmentID", "captureTime", 'stopID'],
    how="inner",
    suffix="_pred"
)

mega_predset = mega_predset.with_columns(
    pl.col('receiveTime').dt.convert_time_zone('America/Chicago'),
)

mega_predset = mega_predset.with_columns(
    (pl.col("time") - pl.col("receiveTime")).alias("predictedDiff")
)

mega_predset = mega_predset.with_columns(
    (pl.duration(minutes=pl.col('minutes'))).alias('minutes')
)

mega_predset = mega_predset.with_columns(
    (pl.col("schedule") - pl.col("receiveTime")).alias("scheduleDiff")
)


mega_predset['routeID', 'equipmentID', 'lat', 'lng', 'captureTime', 'stopID', 'lastStopID', 'nextStopID', 'lastStopName', 'nextStopName_actual', 'time', 'status', 'schedule', 'receiveTime', 'predictedDiff', 'timeDiff', 'scheduleDiff', 'minutesDiff', 'minutes', 'eta', 'statusName']

In [ ]:
#Add a uniform column for nextStopPatternID
subset = subset.with_columns(
    pl.col("nextPatternStopID").alias("patternStopID")
)

#Join the subset on the triple key to get the correct prediction
predSet = subset.join(
    preds,
    on=["equipmentID", "patternStopID", "captureTime"],
    how="inner"
)

predSet = predSet.with_columns(
    pl.col('receiveTime').dt.convert_time_zone('America/Chicago'),
)

#Subtract receiveTime and scheduleTime to get the time difference
predSet = predSet.with_columns(
    (pl.col("time") - pl.col("receiveTime")).alias("predictedDiff")
)

predSet = predSet.with_columns(
    (pl.col("schedule") - pl.col("receiveTime")).alias("scheduleDiff")
)

predSet = predSet.with_columns(
    (pl.col('predictedDiff') - pl.col('timeDiff')).alias('predictedActualDiff')
)

predSet = predSet.with_columns(
    (pl.duration(minutes=pl.col('minutes'))).alias('minutes')
)

predSet = predSet.with_columns(
    (pl.col('receiveTime') + (pl.col('minutes'))).alias('receiveTimePlusMinutes')
)

predSet = predSet.with_columns(
    (pl.col('time') - pl.col('receiveTimePlusMinutes')).alias('receivePlusMinuteDiff')
) 

predSet = predSet.with_columns(
    (pl.col('timeDiff') - pl.col('minutes')).alias('minutesDiff')
)

#Filter out negative schedule differences
# predSet = predSet.filter(
#     (pl.col("scheduleDiff").dt.total_minutes() >= 0) 
# )

predSet['routeID', 'equipmentID', 'lat', 'lng', 'scheduleNumber', 'captureTime', 'stopID', 'lastStopID', 'nextStopID', 'lastStopName', 'nextStopName', 'time', 'status', 'schedule', 'receiveTime', 'receiveTimePlusMinutes', 'receivePlusMinuteDiff', 'predictedDiff', 'timeDiff', 'scheduleDiff', 'predictedActualDiff', 'minutesDiff', 'minutes', 'statusName']


In [ ]:
r2_predset = predSet.filter(
    (pl.col("routeID") == "2L") &
    (pl.col("timeDiff") < pl.duration(minutes=20)) &
    (pl.col("predictedDiff") >= pl.duration(minutes=0))
)

In [ ]:
#Calcuate Metrics
rmse = np.sqrt(mean_squared_error(r2_predset['predictedDiff'].to_numpy(), r2_predset['timeDiff'].to_numpy()))
mae = mean_absolute_error(r2_predset['predictedDiff'].to_numpy(), r2_predset['timeDiff'].to_numpy())
r2 = r2_score(r2_predset['predictedDiff'].to_numpy(), r2_predset['timeDiff'].to_numpy())
print(f"R2 Score: {r2}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

In [ ]:
r2_predset['predictedDiff'].describe()

In [ ]:
predSet['scheduleDiff'].describe()

In [ ]:
predSet['predictedActualDiff'].describe()

In [ ]:
diffSet = predSet.filter(pl.col('scheduleDiff') < pl.duration(minutes=-30))

diffSet['equipmentID', 'routeID', 'lastStopName', 'nextStopName', 'receiveTime', 'captureTime', 'schedule', 'time', 'scheduleDiff', 'timeDiff', 'predictedDiff', 'minutesDiff', 'predictedActualDiff', 'minutes']

In [ ]:
outset = predSet.filter(pl.col("predictedDiff") < pl.duration(seconds=-30))

outset['equipmentID', 'routeID', 'receiveTime', 'captureTime', 'schedule', 'time', 'timeDiff', 'predictedDiff', 'minutesDiff', 'predictedActualDiff', 'minutes']

In [ ]:
d = pl.lit(str(datetime.date.today()))

data = preds[:2000]

data

In [ ]:
#Check if the time and status time match or are on time
#The Dateframe should be empty
preds.filter(
    (pl.col("time") != pl.col("status")) &
    (pl.col("status") != "On Time")
)

In [ ]:
#Check if all the status color change togther
(preds.group_by("equipmentID", "captureTime")).agg(pl.col("statuscolor").n_unique())['statuscolor'].value_counts()

In [ ]:
predSet['scheduleDiff', 'statusName'].describe()

In [ ]:
#Check if our statusName column is correct by comparing it to the status column
data.group_by("statusName").agg(
    pl.col("scheduleDiff").min().alias("scheduleDiffMin"),
    pl.col("scheduleDiff").max().alias("scheduleDiffMax"),
    pl.col("scheduleDiff").mean().alias("scheduleDiffMean"),
    pl.col("scheduleDiff").median().alias("scheduleDiffMedian"),
)

In [ ]:
#Pull data for on time and little late statuses
on_time_data = data.filter(pl.col('statusName') == 'On Time')
little_late_data = data.filter(pl.col('statusName') == 'Little Late')

#Convert the scheduleDiff to scheduleDiff_seconds
on_time_data = on_time_data.with_columns(
    (pl.col("scheduleDiff").dt.total_seconds()).alias("scheduleDiff_seconds") 
)

little_late_data = little_late_data.with_columns(
    (pl.col("scheduleDiff").dt.total_seconds()).alias("scheduleDiff_seconds") 
)


# Plot the histograms
plt.figure(figsize=(15, 6))
plt.hist(on_time_data['scheduleDiff_seconds'].to_list(), bins=50, alpha=0.5, width=5, label='On time Schedule Diff')
plt.hist(little_late_data['scheduleDiff_seconds'].to_list(), bins=50, alpha=0.5, width=5, label='Little Late Schedule Diff')
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency')
plt.title('Histogram of On-time Schedule Diff')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Define a threshold for overlapping data in minutes
threshold = 5.0 * 60

# Filter out pairs where the difference in scheduleDiff_minutes is within the threshold
overlapping_data = data.filter(pl.col('scheduleDiff').dt.total_seconds() >= threshold).sort('scheduleDiff')

overlapping_data['routeID', 'equipmentID', 'receiveTime', 'schedule','statusName', 'scheduleDiff']